# Web Scrapper

- Obtener resultados de google
- Obtener los párrafos de cada resultado
- Preprocesar y guardar

In [61]:
from util.scrapper import *


keywords = ['Larga cola']
data = []
for keyword in keywords:
    try:
        results = scrape_google(keyword, 10, "es")
        for result in results:
            data.append(result)
    except Exception as e:
        print(e)
    finally:
        time.sleep(10)


links = [result['link'] for result in data if is_well_formed_link.match(result['link'])]



In [62]:
import re
is_well_formed_link = re.compile(r'^https?://.+/.+$')
is_root_path = re.compile(r'^/.+$')

def is_useful(string):
    return string != '\n'

In [64]:
import urllib.request
from bs4 import BeautifulSoup
soup = BeautifulSoup(read_content,'html.parser')
for link in links:
    try:
        content = urllib.request.urlopen(link)
        read_content = content.read()
        pAll = soup.find_all('p')
        print(list(filter(is_useful, [p.text for p in pAll])))
    except:
        pass

nstrucción que está dejando de ser simplemente un supuesto teórico para consolidarse como una alternativa de alto valor para las industrias culturales.', '¿Qué propone y por qué es disruptiva la teoría de la Larga Cola?', 'La teoría de la Larga Cola fue formulada en 2004 por Chris Anderson, periodista, escritor, físico y editor de la revista Wired, y posteriormente publicada en 2007 en su libro La economía Long Tail: De Los Mercados De Masas Al Triunfo De Lo Minoritario. ', ' “la cultura y economía digital está dejando de estar concentrada en un número pequeño de ‘éxitos’ (productos y mercados de amplia aceptación) en la cima de la curva de demanda, a dedicarse a una enorme cantidad de nichos especializados en la cola”.', '“Al disminuirse los costos de producción y distribución, especialmente online, ahora hay menos necesidad de abarrotar productos y consumidores en contenedores de talla única. En una era sin las limitaciones del espacio físico de las estanterías y otros cuellos de bot